In [1]:
import fastf1
import pandas as pd
import numpy as np

In [2]:
def process_race_dataframe(df):
    # Convertir todos los tiempos a segundos
    df['Time'] = df['Time'].dt.total_seconds()

    # Establecer el tiempo del ganador en 0
    df.loc[df['Position'] == 1, 'Time'] = 0

    # Renombrar la columna 'Time' a 'Gap to Winner'
    df.rename(columns={'Time': 'Gap to Winner'}, inplace=True)

    return df

In [3]:
# Create a list to store each race's DataFrame
races = []

# Iterate over the first 5 rounds (1 to 5)
for round_number in range(1, 6):
    # Load the race session
    session = fastf1.get_session(2025, round_number, 'R')
    session.load()

    # Select the relevant columns
    df = session.results[['DriverNumber', 'Abbreviation', 'TeamName', 'FullName', 'Position', 'GridPosition', 'Time', 'Points']].copy()

    # Calculate the position gain/loss
    df['PositionGainLoss'] = df['GridPosition'] - df['Position']

    # Add additional columns for race metadata
    df['RaceRound'] = round_number
    df['RaceName'] = session.event['EventName']
    
    # Process the DataFrame (apply transformations like converting Time)
    df = process_race_dataframe(df)

    # Append the DataFrame to the list
    races.append(df)

req         WARNING 	DEFAULT CACHE ENABLED! (777.25 MB) C:\Users\USUARIO\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Fini

In [4]:
# Concatenate all race DataFrames into a single one
final_df = pd.concat(races, ignore_index=True)

In [5]:
final_df

,DriverNumber,Abbreviation,TeamName,FullName,Position,GridPosition,Gap to Winner,Points,PositionGainLoss,RaceRound,RaceName
0,4,NOR,McLaren,Lando Norris,1.0,1.0,0.000,25.0,0.0,1,Australian Grand Prix
1,1,VER,Red Bull Racing,Max Verstappen,2.0,3.0,0.895,18.0,1.0,1,Australian Grand Prix
2,63,RUS,Mercedes,George Russell,3.0,4.0,8.481,15.0,1.0,1,Australian Grand Prix
3,12,ANT,Mercedes,Andrea Kimi Antonelli,4.0,16.0,10.135,12.0,12.0,1,Australian Grand Prix
4,23,ALB,Williams,Alexander Albon,5.0,6.0,12.773,10.0,1.0,1,Australian Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...
95,18,STR,Aston Martin,Lance Stroll,16.0,16.0,5.527,0.0,0.0,5,Saudi Arabian Grand Prix
96,7,DOO,Alpine,Jack Doohan,17.0,17.0,19.264,0.0,0.0,5,Saudi Arabian Grand Prix
97,5,BOR,Kick Sauber,Gabriel Bortoleto,18.0,20.0,19.306,0.0,2.0,5,Saudi Arabian Grand Prix
98,22,TSU,Red Bull Racing,Yuki Tsunoda,19.0,8.0,NaN,0.0,-11.0,5,Saudi Arabian Grand Prix


In [6]:
column_order = ['RaceRound', 'RaceName', 'DriverNumber', 'Abbreviation', 'FullName', 'TeamName', 'Position', 'GridPosition', 'PositionGainLoss','Gap to Winner', 'Points']
final_df = final_df[column_order]

In [7]:
final_df

,RaceRound,RaceName,DriverNumber,Abbreviation,FullName,TeamName,Position,GridPosition,PositionGainLoss,Gap to Winner,Points
0,1,Australian Grand Prix,4,NOR,Lando Norris,McLaren,1.0,1.0,0.0,0.000,25.0
1,1,Australian Grand Prix,1,VER,Max Verstappen,Red Bull Racing,2.0,3.0,1.0,0.895,18.0
2,1,Australian Grand Prix,63,RUS,George Russell,Mercedes,3.0,4.0,1.0,8.481,15.0
3,1,Australian Grand Prix,12,ANT,Andrea Kimi Antonelli,Mercedes,4.0,16.0,12.0,10.135,12.0
4,1,Australian Grand Prix,23,ALB,Alexander Albon,Williams,5.0,6.0,1.0,12.773,10.0
...,...,...,...,...,...,...,...,...,...,...,...
95,5,Saudi Arabian Grand Prix,18,STR,Lance Stroll,Aston Martin,16.0,16.0,0.0,5.527,0.0
96,5,Saudi Arabian Grand Prix,7,DOO,Jack Doohan,Alpine,17.0,17.0,0.0,19.264,0.0
97,5,Saudi Arabian Grand Prix,5,BOR,Gabriel Bortoleto,Kick Sauber,18.0,20.0,2.0,19.306,0.0
98,5,Saudi Arabian Grand Prix,22,TSU,Yuki Tsunoda,Red Bull Racing,19.0,8.0,-11.0,NaN,0.0


In [8]:
final_df.to_csv('f1_2025_season_results.csv', index=False)